In [2]:
import os
from aana.sdk import AanaSDK

from aana.deployments.aana_deployment_handle import AanaDeploymentHandle

os.environ["CUDA_VISIBLE_DEVICES"] = "4"

aana_app = AanaSDK()
aana_app.connect(show_logs=True)

from aana.deployments.face_detection_deployment import (
    FaceDetectorConfig,
    FaceDetectorDeployment,
)

from aana.deployments.face_featureextraction_deployment import (
    FacefeatureExtractorConfig,
    FacefeatureExtractorDeployment,
)

face_detector_deployment = FaceDetectorDeployment.options(
    num_replicas=1,
    ray_actor_options={"num_gpus": 0.1},
    user_config=FaceDetectorConfig(
        model_url="/nas/dominic/Production/FaceModel_REST/InsightFace-REST/models/onnx/scrfd_10g_gnkps/scrfd_10g_gnkps.onnx",
        nms_thresh=0.4,
        batch_size=4,
        input_size=640,
    ).model_dump(mode="json"),
)

aana_app.register_deployment("face_detector", face_detector_deployment, deploy=True)


handle = await AanaDeploymentHandle.create("face_detector")


FACEFEATURE_MODEL = "ir_101_debug"

facefeat_extractor_deployment = FacefeatureExtractorDeployment.options(
    num_replicas=1,
    ray_actor_options={"num_gpus": 0.5},
    user_config=FacefeatureExtractorConfig(
        feature_extractor_name=FACEFEATURE_MODEL,
        min_face_norm=19.0,
    ).model_dump(mode="json"),
)


aana_app.register_deployment(
    "facefeat_extractor", facefeat_extractor_deployment, deploy=True
)


facefeat_handle = await AanaDeploymentHandle.create("facefeat_extractor")

/root/.cache/pypoetry/virtualenvs/aana-vIr3-B0u-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-08 10:59:44,014	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
/root/.cache/pypoetry/virtualenvs/aana-vIr3-B0u-py3.10/lib/python3.10/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_dir" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ('settings_',)`.
  warnings.warn(
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
(ProxyActor pid=458255) INFO 2024-07-08 10:59:52,925 proxy 172.17.0.3 proxy.py:1161 - Pro

(ServeReplica:face_detector:FaceDetectorDeployment pid=459453) Available providers: ['CPUExecutionProvider']


(ServeReplica:face_detector:FaceDetectorDeployment pid=459453) INFO 2024-07-08 10:59:58,292 face_detector_FaceDetectorDeployment wc9gvpj4 1e9a9fbc-7de3-473d-aefc-03cbd2283dc2 replica.py:373 - GET_METHODS OK 4.7ms
(ServeController pid=458168) INFO 2024-07-08 10:59:58,401 controller 458168 deployment_state.py:1598 - Deploying new version of Deployment(name='FacefeatureExtractorDeployment', app='facefeat_extractor') (initial target replicas: 1).
(ServeController pid=458168) INFO 2024-07-08 10:59:58,507 controller 458168 deployment_state.py:1844 - Adding 1 replica to Deployment(name='FacefeatureExtractorDeployment', app='facefeat_extractor').


(ServeReplica:facefeat_extractor:FacefeatureExtractorDeployment pid=459571) INFO 2024-07-08 11:00:05,420 facefeat_extractor_FacefeatureExtractorDeployment mwckl3pl 189bc160-0714-4435-8164-23b624429bf4 replica.py:373 - GET_METHODS OK 5.8ms
(raylet) [2024-07-08 11:00:07,767 E 454540 454578] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-07-08_10-59-45_449295_454210 is over 95% full, available space: 20528304128; capacity: 983347249152. Object creation will fail if spilling is required.
(ServeReplica:face_detector:FaceDetectorDeployment pid=459453) INFO 2024-07-08 11:00:09,934 face_detector_FaceDetectorDeployment wc9gvpj4 5c473aa7-5b5c-48ea-99aa-95aa9c25cc07 replica.py:373 - PREDICT OK 603.8ms
(ServeReplica:facefeat_extractor:FacefeatureExtractorDeployment pid=459571) INFO 2024-07-08 11:00:10,482 facefeat_extractor_FacefeatureExtractorDeployment mwckl3pl e2e901cd-8aab-43bc-b063-32e21107a75d replica.py:373 - PREDICT OK 539.4ms
(raylet) [2024-07-08 11:00:17,780 E 454540 454578] 

In [3]:
from aana.core.models.image import Image
from pathlib import Path

image = Image(
    path=Path(
        "/nas/datasets/CondensedMoviesLite/AllFrames_3fps/-1gCG8m1SHU/000000101.jpg"
    )
)

detector_output = await handle.predict([image])

# print(detector_output)

keypoints = detector_output["keypoints"]
facefeat_output = await facefeat_handle.predict([image], keypoints)

print(facefeat_output)

{'facefeats_per_image': [{'face_feats': array([[-0.04401124, -0.0272526 ,  0.01495183, ..., -0.06465618,
         0.0389987 ,  0.0128512 ],
       [ 0.01096947, -0.02446904,  0.03266014, ..., -0.01851118,
         0.01181385,  0.02578571]], dtype=float32), 'norms': array([[21.607306],
       [23.78863 ]], dtype=float32)}]}


In [4]:
from aana.configs.settings import settings
import importlib
import aana.deployments.face_database_deployment

importlib.reload(aana.deployments.face_database_deployment)
from aana.deployments.face_database_deployment import (
    FaceDatabaseConfig,
    FaceDatabaseDeployment,
)


facedatabase_config = FaceDatabaseConfig(
    face_threshold=1.18,
    face_features_directory=settings.artifacts_dir / "face_features_database",
    feature_extractor_name=FACEFEATURE_MODEL,
)
facedatabase_deployment = FaceDatabaseDeployment()
await facedatabase_deployment.apply_config(facedatabase_config.model_dump(mode="json"))

In [5]:
query_feats = facefeat_output['facefeats_per_image'][0]['face_feats']

In [11]:
res = await facedatabase_deployment.search(query_feats)

In [13]:
facefeat_output['facefeats_per_image']

[{'face_feats': array([[-0.04401124, -0.0272526 ,  0.01495183, ..., -0.06465618,
           0.0389987 ,  0.0128512 ],
         [ 0.01096947, -0.02446904,  0.03266014, ..., -0.01851118,
           0.01181385,  0.02578571]], dtype=float32),
  'norms': array([[21.607306],
         [23.78863 ]], dtype=float32)}]

In [9]:
import numpy as np
facefeats_ir101_web4M = np.load('/nas/dominic/AanaFaceEval/reference_facedict_aanasdk_webface4M_r101.npy', allow_pickle=True).item()

In [35]:
facefeats_ir101_web4M['face_features']

array([ 3.75524117e-03,  9.27040447e-03,  1.37306917e-02, -2.38199215e-02,
       -6.22658171e-02,  3.70729826e-02, -2.66999062e-02,  5.63183846e-03,
       -4.03649956e-02,  2.73555126e-02,  8.26491341e-02,  9.14010778e-03,
       -7.89554566e-02,  6.68178424e-02, -2.72627827e-02, -2.07748506e-02,
       -3.96570042e-02,  3.30630131e-02, -3.81770581e-02,  1.46513162e-02,
       -3.37886550e-02,  3.04316562e-02,  8.05255864e-03, -1.82717778e-02,
       -2.65709087e-02, -7.82726184e-02, -3.83070707e-02, -5.22765040e-04,
       -9.14224051e-03,  1.60650499e-02, -2.02019364e-02,  7.69493654e-02,
       -2.45333090e-03, -5.97015321e-02,  2.04053763e-02, -1.52246347e-02,
       -2.44577192e-02, -2.13037059e-02, -3.20889726e-02,  5.80648184e-02,
       -2.03799382e-02, -4.44113277e-02, -5.00523299e-02, -9.89137590e-03,
        6.89015761e-02, -4.69854549e-02,  1.16573973e-03,  2.46160459e-02,
       -4.66552787e-02, -4.65669930e-02, -1.04997996e-02,  4.72796820e-02,
        3.59151028e-02, -

In [10]:
import tqdm

for k in tqdm.tqdm(range(len(facefeats_ir101_web4M['face_features']))):
    ffeat = facefeats_ir101_web4M['face_features'][k]
    name_ = facefeats_ir101_web4M['person_names'][k]
    img_id = facefeats_ir101_web4M['paths_to_image'][k].split('|||')[0].split('/')[-1]

    res = await facedatabase_deployment.add_reference_face(
    ffeat, person_name=name_, image_id=img_id
)

100%|██████████| 32898/32898 [07:44<00:00, 70.85it/s]  


In [39]:
facefeats_ir101_web4M['paths_to_image'][0].split('|||')[0].split('/')[-1]

'000744f4-4131-44db-a569-ca211fa55a48'

In [20]:
import numpy as np
face_feature = np.expand_dims(ffeat, axis=0).astype("float32")

In [22]:
from aana.configs.settings import settings
from aana.deployments.face_database_deployment import (
    FaceDatabaseConfig,
    FaceDatabaseDeployment,
)

facedatabase_deployment = FaceDatabaseDeployment.options(
    num_replicas=1,
    ray_actor_options={"num_gpus": 0.1},
    user_config=FaceDatabaseConfig(
        face_threshold=1.18,
        face_features_directory=settings.artifacts_dir / "face_features_database",
        feature_extractor_name=FACEFEATURE_MODEL,
    ).model_dump(mode="json"),
)

aana_app.register_deployment(
    "facedatabase_deployment", facedatabase_deployment, deploy=True
)

AttributeError: type object 'FaceDatabaseDeployment' has no attribute 'options'

In [1]:
facedatabase_handle = await AanaDeploymentHandle.create("facedatabase_deployment")

NameError: name 'AanaDeploymentHandle' is not defined

In [45]:
# Sample image paths
image_paths = [
    "image1.jpg",
    "image2.jpg",
    "image3.jpg",
    "image4.jpg",
    "image5.jpg",
    "image6.jpg",
    "image7.jpg",
    "image8.jpg",
    "image9.jpg",
    "image10.jpg",
]

# Populate the data dictionary
data = {"images": [{"path": im_path} for im_path in image_paths[0:10]]}

# Print the result
print(data)

{'images': [{'path': 'image1.jpg'}, {'path': 'image2.jpg'}, {'path': 'image3.jpg'}, {'path': 'image4.jpg'}, {'path': 'image5.jpg'}, {'path': 'image6.jpg'}, {'path': 'image7.jpg'}, {'path': 'image8.jpg'}, {'path': 'image9.jpg'}, {'path': 'image10.jpg'}]}


In [49]:
import requests, json


def extract_face_feats(im_path):
    data = {
        "images": [{"path": im_path}, {"path": im_path}],
    }

    response = requests.post(
        "http://127.0.0.1:8000/recognize_faces",
        data={"body": json.dumps(data)},
        stream=False,
    )
    res = response.json()

    return res
    # return res['face_features_per_image'][0]['face_feats'], res['face_features_per_image'][0]['norms']


def extract_face_feats_batched(im_paths):
    data = {"images": [{"path": im_path} for im_path in im_paths]}

    print(data)

    response = requests.post(
        "http://127.0.0.1:8000/recognize_faces",
        data={"body": json.dumps(data)},
        stream=False,
    )
    res = response.json()

    return res

    # face_feats = [res['face_features_per_image'][k]['face_feats'] for k in range(len(res['face_features_per_image']))]
    # norms = [res['face_features_per_image'][k]['norms'] for k in range(len(res['face_features_per_image']))]

    # return face_feats, norms

In [40]:
res = extract_face_feats(
    "/nas/dominic/AanaFaceEval/identities_reference/default/000bb305-88c1-4490-977f-59e66b755cd4|||Ilse Neubauer.jpg"
)

In [55]:
data = {"images": [{"path": im_path} for im_path in image_paths[0:10]]}

print(data)

{'images': [{'path': '/nas/dominic/AanaFaceEval/identities_reference/default/000744f4-4131-44db-a569-ca211fa55a48|||Keita Machida.jpg'}, {'path': '/nas/dominic/AanaFaceEval/identities_reference/default/0007970d-a5d9-4791-bb47-9c52d4cf4473|||Caroleen Feeney.jpg'}, {'path': '/nas/dominic/AanaFaceEval/identities_reference/default/0009aaba-e0da-46c3-a557-72d318ce0bd8|||Ewa Fröling.jpg'}, {'path': '/nas/dominic/AanaFaceEval/identities_reference/default/000b66a3-5009-4513-aa13-535202c679b3|||Marielle Heller.jpg'}, {'path': '/nas/dominic/AanaFaceEval/identities_reference/default/000bb305-88c1-4490-977f-59e66b755cd4|||Ilse Neubauer.jpg'}, {'path': '/nas/dominic/AanaFaceEval/identities_reference/default/000d385b-ac12-41b0-9692-7abff6c5546a|||Michel Joelsas.jpg'}, {'path': '/nas/dominic/AanaFaceEval/identities_reference/default/00112729-6837-424c-bdf5-1709b9b38cba|||Patricia Hodge.jpg'}, {'path': '/nas/dominic/AanaFaceEval/identities_reference/default/00115272-41a7-4b22-bb5a-ea079aa2dea6|||Gilli

In [72]:
res = extract_face_feats_batched(image_paths[13:19])

{'images': [{'path': '/nas/dominic/AanaFaceEval/identities_reference/default/001c3bcf-3ab0-48cc-b805-c8da6ed3ba6a|||Eryk Lubos.jpg'}, {'path': '/nas/dominic/AanaFaceEval/identities_reference/default/001d0484-4b83-42da-bff9-145bcc46359b|||Sanjjanaa Archana Galrani.jpg'}, {'path': '/nas/dominic/AanaFaceEval/identities_reference/default/001e04f1-a8f4-45e0-a85d-59d8bf06f12a|||Gerald Okamura.jpg'}, {'path': '/nas/dominic/AanaFaceEval/identities_reference/default/0021297f-e5f7-4965-a0cd-bb42c4d70732|||Laurie Metcalf.jpg'}, {'path': '/nas/dominic/AanaFaceEval/identities_reference/default/00227c4a-8cf5-491f-823d-4bd6251efac1|||Luís Melo.jpg'}, {'path': '/nas/dominic/AanaFaceEval/identities_reference/default/0022e2e0-f72b-4470-af26-0b93527b3ebf|||Marie-Mae van Zuilen.jpg'}]}


In [73]:
res

{'error': 'IndexError',
 'message': 'list index out of range',
 'data': {},
 'stacktrace': '\x1bray::ServeReplica:facefeat_extractor_deployment:FacefeatureExtractorDeployment.handle_request_with_rejection()\x1b (pid=9498, ip=172.17.0.3, actor_id=2d13ca81aac0a71ca1ac728801000000, repr=<ray.serve._private.replica.ServeReplica:facefeat_extractor_deployment:FacefeatureExtractorDeployment object at 0x7f035bcd8af0>)\n    yield await self._user_callable_wrapper.call_user_method(\n  File "/root/.cache/pypoetry/virtualenvs/aana-vIr3-B0u-py3.10/lib/python3.10/site-packages/ray/serve/_private/replica.py", line 1150, in call_user_method\n    raise e from None\nray.exceptions.RayTaskError: \x1bray::ServeReplica:facefeat_extractor_deployment:FacefeatureExtractorDeployment.handle_request_with_rejection()\x1b (pid=9498, ip=172.17.0.3)\n  File "/root/.cache/pypoetry/virtualenvs/aana-vIr3-B0u-py3.10/lib/python3.10/site-packages/ray/serve/_private/utils.py", line 168, in wrap_to_ray_error\n    raise exce

In [2]:
import tqdm
import glob
import os


reference_faces_dict_30K_webface4M_r101 = {
    "person_names": [],
    "paths_to_image": [],
    "face_features": [],
    "norms": [],
}

failed_faces = []

image_paths = sorted(
    glob.glob("/nas/dominic/AanaFaceEval/identities_reference/default/*.jpg")
)

for image_filepath in tqdm.tqdm(image_paths):
    face_feats, norms = extract_face_feats(image_filepath)

    # import pdb; pdb.set_trace()

    if face_feats is not None and len(face_feats) == 1 and norms[0][0] >= 19.0:
        reference_faces_dict_30K_webface4M_r101["person_names"].append(
            os.path.basename(image_filepath)[0:-4].split("|||")[-1]
        )
        reference_faces_dict_30K_webface4M_r101["paths_to_image"].append(image_filepath)
        reference_faces_dict_30K_webface4M_r101["face_features"].append(face_feats[0])
        reference_faces_dict_30K_webface4M_r101["norms"].append(norms[0][0])
    else:
        failed_faces.append([image_filepath, norms[0][0]])

reverse_reference_faces_dict_30K_webface4M_r101 = {
    reference_faces_dict_30K_webface4M_r101["person_names"][
        k
    ]: reference_faces_dict_30K_webface4M_r101["paths_to_image"][k]
    for k in range(len(reference_faces_dict_30K_webface4M_r101["person_names"]))
}

  0%|          | 28/33541 [00:07<2:38:11,  3.53it/s]


KeyboardInterrupt: 